In [ ]:
# ============================================================
# 0) Setup
# ============================================================
!pip install -q rasterio google-cloud-storage imageio torch torchvision requests

import os, re, csv, io, time, json
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import imageio.v2 as imageio
from google.colab import auth
from google.cloud import storage
from collections import defaultdict
from datetime import datetime
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from rasterio.enums import Resampling
import shutil # Import shutil for file operations
import requests # Import requests
from io import BytesIO # Import BytesIO


# Authenticate in your Colab notebook
from google.colab import auth
auth.authenticate_user(clear_output=True)

# ✅ Force ADC and environment variables for GDAL/Rasterio
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/.config/application_default_credentials.json"
os.environ["CLOUDSDK_AUTH_CREDENTIAL_FILE_OVERRIDE"] = "/content/.config/application_default_credentials.json"
os.environ["GS_OAUTH2_TOKEN"] = "ya29.fake"  # bypass internal GDAL auth check
os.environ["CPL_VSIL_CURL_ALLOWED_EXTENSIONS"] = ".tif"
os.environ["GDAL_DISABLE_READDIR_ON_OPEN"] = "EMPTY_DIR"
os.environ["GS_REQUESTER_PAYS"] = "YES"  # optional for project-billed buckets
os.environ["GS_NO_SIGN_REQUEST"] = "NO"


# Run this command to make all files in your bucket readable by anyone:
# !gsutil iam ch allUsers:objectViewer gs://nasa_sar_spacetron # Not needed with authenticated access

# Confirm that it worked:
# !gsutil iam get gs://nasa_sar_spacetron # Not strictly needed for testing authenticated access


# This creates ADC at ~/.config/gcloud/application_default_credentials.json
# !gcloud auth application-default login -q # This is handled by auth.authenticate_user()

# Set GOOGLE_APPLICATION_CREDENTIALS for GDAL/Rasterio
# Note: The path might be slightly different depending on the Colab environment,
# but this is a common location for ADC created by gcloud auth.
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/root/.config/gcloud/application_default_credentials.json" # Redundant with explicit path above


# ============================================================
# 1) Config (EDIT HERE IF NEEDED)
# ============================================================
PROJECT_ID        = "nasa-space-apps-473722"
BUCKET            = "nasa_sar_spacetron"

MOSAICS_DIR       = "CLEAN/SAR_Mosaics"       # baseline mosaics
HOTSPOTS_DIR      = "CLEAN/Interest_clean"    # your hotspot/interest mosaics

# Control window (for “normal/baseline” reference)
CONTROL_START     = (2018, "Q1")   # inclusive
CONTROL_END       = (2019, "Q2")   # inclusive

ALL_YEARS         = list(range(2018, 2025))
QUARTS            = ["Q1","Q2","Q3","Q4"]
AOIS              = None  # None = all AOIs; or [1,2,3]

# Choose the bands you want to use (best practice):
#   Option A (recommended): ["VV_dB","VH_dB","angle"]
#   Option B (if angle not available/consistent): ["VV_dB","VH_dB","VV_minus_VH_dB"]
SELECT_BANDS      = ["VV_dB","VH_dB","angle"]

# Output Directories
# OUT_ROOT_GCS      = "manifest/unified_timeseries"  # stats, plots, gifs, anomaly csv
OUT_DRIVE_DIR     = "/content/drive/MyDrive/SAR_Results" # Output directory in Google Drive
MODEL_DIR_GCS     = "models/trenmaya_unetae/v1_ae" # saved model + metadata

# Plots & GIFs
MAKE_PLOTS        = True
MAKE_GIFS         = True
GIF_DOWNSAMPLE    = 6            # downsample factor for GIF frames (keeps them small)

# Thresholds (soft heuristics; adaptive fallback applied below)
WATER_VV_DB_THR   = -17.0        # typical water VV threshold (dB)
WATER_VH_DB_THR   = -22.0        # typical water VH threshold (dB)
BARE_DIFF_DB_THR  = 6.0          # VV-VH > 6 dB + VH low ⇒ bare/infrastructure
VH_LOW_DB_THR     = -15.0

# ML hyperparams
PATCH_SIZE        = 256
BATCH_SIZE        = 8
EPOCHS_MAX        = 80
PATIENCE          = 12
LR                = 1e-3
USE_AMP           = True
BASE_CHANNELS     = 32
ANOM_PCTL         = 98    # percentile for "interesting pixels"
USE_STREAMING_PATCHES = True  # True = sample patches by windowed reads (memory-safe, slower)

# Define IN_CHANNELS based on SELECT_BANDS
IN_CHANNELS = len(SELECT_BANDS)

# ============================================================
# 2) GCS helpers & file parsing
# ============================================================
client = storage.Client(project=PROJECT_ID)
bucket_obj = client.bucket(BUCKET)

def list_tifs(prefix):
    return [b.name for b in client.list_blobs(BUCKET, prefix=prefix) if b.name.endswith(".tif")]

# Patterns like: AOI_1_2020_Q1.tif  / AOI 1 2020 Q1.tif
PAT = re.compile(r".*AOI[_ ]?(\d+)[_ ]?(\d{4})[_ ]?Q([1-4])\.tif$", re.IGNORECASE)
def parse_info(path):
    m = PAT.match(os.path.basename(path))
    if not m: return None
    return dict(aoi=int(m.group(1)), year=int(m.group(2)), quart=f"Q{m.group(3)}", path=path)

def list_items():
    items = []
    for src in (MOSAICS_DIR, HOTSPOTS_DIR):
        for p in list_tifs(src):
            meta = parse_info(p)
            if meta:
                meta["source"] = "hotspot" if src == HOTSPOTS_DIR else "baseline"
                items.append(meta)
    if AOIS:
        items = [x for x in items if x["aoi"] in AOIS]
    return sorted(items, key=lambda d:(d["aoi"], d["year"], d["quart"], d["source"]))

items = list_items()
assert items, "No mosaics found. Check folder names and file patterns."
print(f"Found {len(items)} rasters across baseline + hotspots (first 5):")
print(items[:5])

# Useful dicts
orderQ = {"Q1":1, "Q2":2, "Q3":3, "Q4":4}
def t_index(y,q): return y*4 + orderQ[q]
def in_control(y,q):
    return t_index(*CONTROL_START) <= t_index(y,q) <= t_index(*CONTROL_END)

# Replace your helper function:
def vsicurl_path(gcs_relpath):
    """
    Return a public HTTPS link (no authentication) for rasterio to read.
    """
    if gcs_relpath.startswith("gs://"):
        parts = gcs_relpath.replace("gs://", "").split("/", 1)
        bucket, path = parts[0], parts[1]
    else:
        bucket, path = BUCKET, gcs_relpath
    return f"/vsicurl/https://storage.googleapis.com/{bucket}/{path}"


# ============================================================
# 3) Reading + band selection + basic proxies
# ============================================================
# Replace your read_selected_bands function:
def read_selected_bands(gcs_path, select_names=SELECT_BANDS):
    """Read TIFF from GCS URL, using fallback mapping when band names are missing."""
    try:
        # Build the HTTPS path
        if gcs_path.startswith("gs://"):
            parts = gcs_path.replace("gs://", "").split("/", 1)
            bucket, path = parts[0], parts[1]
            http_url = f"https://storage.googleapis.com/{bucket}/{path}"
        elif gcs_path.startswith("CLEAN/"):
            http_url = f"https://storage.googleapis.com/{BUCKET}/{gcs_path}"
        else:
            http_url = f"https://storage.googleapis.com/{BUCKET}/{gcs_path}"

        # Download and open with rasterio
        response = requests.get(http_url, stream=True)
        response.raise_for_status()
        file_content = BytesIO(response.content)

        with rasterio.open(file_content) as src:
            arr = src.read()
            prof = src.profile
            desc = list(src.descriptions or [])
            desc = [d if d else f"B{i+1}" for i, d in enumerate(desc)]

        # --- Decide how to map bands ---
        n_bands = arr.shape[0]
        if not any("VV" in d or "VH" in d for d in desc):
            # No names → assume default mapping
            # (many Sentinel-1 mosaics follow: B1=VV, B2=VH, B3=angle)
            mapping = {"VV_dB": 0, "VH_dB": 1, "angle": 2}
            print(f"ℹ️ Using default band mapping (B1→VV, B2→VH, B3→angle) for {os.path.basename(http_url)}")
        else:
            # Try to find matching names
            mapping = {}
            for name in select_names:
                found = [i for i, d in enumerate(desc) if name.lower() in d.lower()]
                if found:
                    mapping[name] = found[0]
                else:
                    mapping[name] = None

        # --- Assemble selected array ---
        selected_arr = []
        for name in select_names:
            idx = mapping.get(name, None)
            if idx is None or idx >= n_bands:
                # Use zero-filled fallback if not found
                selected_arr.append(np.zeros_like(arr[0]))
            else:
                selected_arr.append(arr[idx])

        selected_arr = np.stack(selected_arr, axis=0)
        return selected_arr.astype("float32"), prof, desc

    except requests.exceptions.RequestException as e:
        print(f"❌ HTTP error downloading {gcs_path}: {e}")
        return None, None, None
    except Exception as e:
        print(f"❌ Error reading {gcs_path}: {e}")
        return None, None, None


# Replace your compute_proxies() function:
def compute_proxies(vv_db, vh_db):
    """Compute SAR proxies (RVI, PR, water/bare fractions) robustly."""
    # Replace NaNs and infs with sensible defaults
    vv_db = np.nan_to_num(vv_db, nan=-9999.0, posinf=-9999.0, neginf=-9999.0)
    vh_db = np.nan_to_num(vh_db, nan=-9999.0, posinf=-9999.0, neginf=-9999.0)

    if np.all(vv_db == -9999.0) or np.all(vh_db == -9999.0):
        # No valid pixels at all
        return None, None, None, None

    vv_lin = db_to_lin(vv_db)
    vh_lin = db_to_lin(vh_db)

    # --- RVI and PR ---
    rvi = np.zeros_like(vv_lin)
    valid = (vv_lin + vh_lin) > 1e-6
    rvi[valid] = 4.0 * vh_lin[valid] / (vv_lin[valid] + vh_lin[valid])
    rvi = np.clip(rvi, 0, 1)

    pr = np.zeros_like(vv_lin)
    valid = vv_lin > 1e-6
    pr[valid] = vh_lin[valid] / vv_lin[valid]
    pr = np.clip(pr, 0, 3)

    # --- Water / Bare masks ---
    vv_thr = WATER_VV_DB_THR if np.isfinite(WATER_VV_DB_THR) else np.nanpercentile(vv_db, 7)
    vh_thr = WATER_VH_DB_THR if np.isfinite(WATER_VH_DB_THR) else np.nanpercentile(vh_db, 7)
    vv_thr = vv_thr if np.isfinite(vv_thr) else -20
    vh_thr = vh_thr if np.isfinite(vh_thr) else -25

    water_mask = (vv_db <= vv_thr) & (vh_db <= vh_thr)
    diff_db = vv_db - vh_db
    bare_mask = (diff_db >= BARE_DIFF_DB_THR) & (vh_db <= VH_LOW_DB_THR)

    metrics = {
        "vv_db_mean": float(np.nanmean(vv_db)),
        "vh_db_mean": float(np.nanmean(vh_db)),
        "rvi_mean": float(np.nanmean(rvi)),
        "pr_mean": float(np.nanmean(pr)),
        "water_frac": float(np.nanmean(water_mask)),
        "bare_frac": float(np.nanmean(bare_mask)),
    }

    # Skip scenes with all NaNs or all zeros
    if not np.isfinite(metrics["rvi_mean"]) or np.isnan(metrics["rvi_mean"]):
        return None, None, None, None

    return metrics, rvi, water_mask, bare_mask



# ============================================================
# 4) Iterate images → metrics per AOI/time; save CSV
# ============================================================
records = []
by_aoi_time = defaultdict(dict)   # (aoi, t_index) -> metrics

print("Computing metrics per raster...")
for it in items:
    try:
        arr, prof, desc = read_selected_bands(it["path"], SELECT_BANDS)
        if arr is None or arr.shape[0] < 2:
            print(f"⚠️ Skipping {it['path']} — unreadable or missing bands.")
            continue

        # Expect at least VV & VH in dB in the first two positions based on SELECT_BANDS
        if arr.shape[0] < 2:
            print(f"Skipping {it['path']}: Less than 2 bands read.")
            continue

        vv_db = arr[0]; vh_db = arr[1]
        result = compute_proxies(vv_db, vh_db)
        if result[0] is None:
            print(f"⚠️ Skipping {it['path']} — no valid pixels.")
            continue

        metrics, rvi, water_mask, bare_mask = result
        rec = dict(
            aoi=it["aoi"], year=it["year"], quart=it["quart"], source=it["source"],
            t_index=t_index(it["year"], it["quart"]),
            path=it["path"], **metrics
        )
        records.append(rec)
        by_aoi_time[(it["aoi"], rec["t_index"])] = rec
    except Exception as e:
        # Catch any other unexpected errors during the rest of the processing loop
        print(f"❌ Error processing {it['path']}: {e}")
        continue


if not records:
    raise ValueError("No valid raster files processed. Please check input paths and file formats.")

# Save CSV to Drive
os.makedirs("/content/out", exist_ok=True) # Keep local staging dir if needed
os.makedirs(OUT_DRIVE_DIR, exist_ok=True) # Ensure Drive output directory exists
csv_path = "/content/out/sar_metrics.csv"
with open(csv_path, "w", newline="") as f:
    w = csv.DictWriter(f, fieldnames=list(records[0].keys()))
    w.writeheader(); w.writerows(records)

# Replace gcs_upload with save_to_drive
def save_to_drive(local_path, filename=None):
    if not filename:
        filename = os.path.basename(local_path)
    dest = os.path.join(OUT_DRIVE_DIR, filename)
    shutil.copy(local_path, dest)
    print(f"✅ Saved to Drive: {dest}")

save_to_drive(csv_path, "sar_metrics.csv")
print("✅ Metrics CSV saved to Drive.")

# ============================================================
# 5) Recovery analytics + narrative per AOI  (robust version)
# ============================================================
def aois():
    return sorted(set(r["aoi"] for r in records))

def rows_for_aoi(aoi):
    return sorted([r for r in records if r["aoi"] == aoi], key=lambda d: d["t_index"])

def control_rows(rows):
    return [r for r in rows if in_control(r["year"], r["quart"])]

def period_rows(rows, start_year, end_year):
    return [r for r in rows if start_year <= r["year"] <= end_year]

def latest_in_year(rows, year):
    cands = [r for r in rows if r["year"] == year]
    return sorted(cands, key=lambda d: d["t_index"])[-1] if cands else None

def compute_recovery_series(rows, key, decline_start=2018, decline_end=2022, final_year=2025):
    """Compute recovery metrics robustly (ignores NaN)."""
    vals_ctrl = [r[key] for r in control_rows(rows) if np.isfinite(r.get(key, np.nan))]
    vals_decl = [r[key] for r in period_rows(rows, decline_start, decline_end) if np.isfinite(r.get(key, np.nan))]
    last_r = latest_in_year(rows, final_year)

    baseline = np.nanmean(vals_ctrl) if vals_ctrl else np.nan
    worst = np.nanmin(vals_decl) if vals_decl else np.nan
    final_val = float(last_r[key]) if last_r and np.isfinite(last_r.get(key, np.nan)) else np.nan

    denom = baseline - worst
    if not np.isfinite(baseline) or not np.isfinite(worst) or not np.isfinite(final_val) or abs(denom) < 1e-9:
        rec_pct = np.nan
    else:
        rec_pct = np.clip((final_val - worst) / denom * 100.0, -200, 200)

    return dict(baseline=baseline, worst=worst, final=final_val, recovery_pct=rec_pct)

summaries = []
for a in aois():
    rows = rows_for_aoi(a)
    if not rows:
        continue

    rvi = compute_recovery_series(rows, "rvi_mean")
    wat = compute_recovery_series(rows, "water_frac")
    bare = compute_recovery_series(rows, "bare_frac")

    # Skip AOIs completely empty
    if all(np.isnan(x["baseline"]) for x in [rvi, wat, bare]):
        print(f"⚠️ AOI {a} has no valid data — skipped.")
        continue

    summaries.append(dict(
        aoi=a,
        rvi_baseline=rvi["baseline"], rvi_worst=rvi["worst"], rvi_2025=rvi["final"], rvi_recovery_pct=rvi["recovery_pct"],
        water_baseline=wat["baseline"], water_worst=wat["worst"], water_2025=wat["final"], water_recovery_pct=wat["recovery_pct"],
        bare_baseline=bare["baseline"], bare_worst=bare["worst"], bare_2025=bare["final"], bare_recovery_pct=bare["recovery_pct"]
    ))

# --- Always write a CSV, even if some AOIs missing ---
os.makedirs("/content/out", exist_ok=True)
sum_path = "/content/out/recovery_summaries.csv"
if summaries:
    with open(sum_path, "w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=list(summaries[0].keys()))
        w.writeheader(); w.writerows(summaries)
    save_to_drive(sum_path, "recovery_summaries.csv")
    print(f"✅ Recovery summaries saved to Drive ({len(summaries)} AOIs).")
else:
    # Create empty CSV with headers for consistency
    with open(sum_path, "w", newline="") as f:
        f.write("aoi,rvi_baseline,rvi_worst,rvi_2025,rvi_recovery_pct,water_baseline,water_worst,water_2025,water_recovery_pct,bare_baseline,bare_worst,bare_2025,bare_recovery_pct\n")
    save_to_drive(sum_path, "recovery_summaries_empty.csv")
    print("⚠️ No valid AOIs for recovery summaries — empty CSV created.")


# ============================================================
# 6) Plots per AOI (timelines for RVI, water, bare)
# ============================================================
if MAKE_PLOTS:
    os.makedirs("/content/out/plots", exist_ok=True)
    for a in aois():
        rows = rows_for_aoi(a)
        rows = [r for r in rows if r["aoi"] in aois() and r["t_index"] in by_aoi_time.keys()] # Filter for successfully processed records
        if not rows: continue # Skip AOI if no records were processed

        rows = sorted(rows, key=lambda d:d["t_index"])
        xs = [f"{r['year']}_{r['quart']}" for r in rows]
        rvi = [r["rvi_mean"] for r in rows]
        wat = [r["water_frac"] for r in rows]
        bar = [r["bare_frac"] for r in rows]
        col = ["#d62728" if r["source"]=="hotspot" else "#1f77b4" for r in rows]

        fig, ax = plt.subplots(3, 1, figsize=(12,8), sharex=True)
        ax[0].plot(xs, rvi, marker="o"); ax[0].set_ylabel("RVI (veg/biomass)")
        ax[1].plot(xs, wat, marker="o"); ax[1].set_ylabel("Water fraction")
        ax[2].plot(xs, bar, marker="o"); ax[2].set_ylabel("Bare/infra fraction")
        for a_i in ax:
            a_i.grid(True, alpha=0.3)
        plt.xticks(rotation=60)
        plt.suptitle(f"AOI {a} — baseline (blue) vs hotspots (red)")
        plt.tight_layout(rect=[0,0,1,0.98])

        png = f"/content/out/plots/ao{a}_timeseries.png"
        plt.savefig(png, dpi=170); plt.close()
        # Replace gcs_upload with save_to_drive
        save_to_drive(png)


# ============================================================
# 7) Optional GIF per AOI (RVI map through time, low-res)
# ============================================================
def make_rvi_frame(vv_db, vh_db, ds=GIF_DOWNSAMPLE):
    vv_lin = db_to_lin(vv_db); vh_lin = db_to_lin(vh_db)
    rvi = 4.0 * vh_lin / np.maximum(vv_lin + vh_lin, 1e-6)
    rvi = np.clip(rvi, 0, 1)
    if ds and ds>1:
        H,W = rvi.shape
        rvi = rvi[:H - H%ds, :W - W%ds]   # crop
        rvi = rvi.reshape(rvi.shape[0]//ds, ds, rvi.shape[1]//ds, ds).mean(axis=(1,3))
    # stretch to 0..255
    lo, hi = np.nanpercentile(rvi, 2), np.nanpercentile(rvi, 98)
    v = np.clip((rvi - lo) / max(1e-6, hi-lo), 0, 1)
    return (v*255).astype(np.uint8)

if MAKE_GIFS:
    os.makedirs("/content/out/gifs", exist_ok=True)
    for a in aois():
        rows = rows_for_aoi(a)
        rows = [r for r in rows if r["aoi"] in aois() and r["t_index"] in by_aoi_time.keys()] # Filter for successfully processed records
        if not rows: continue # Skip AOI if no records were processed

        frames = []; labels=[]
        for r in sorted(rows, key=lambda d:d["t_index"]):
            try:
                arr, _, _ = read_selected_bands(r["path"], SELECT_BANDS)
                if arr is None: continue # Skip if reading failed

                # Ensure we have at least two bands for VV and VH
                if arr.shape[0] < 2:
                    print(f"Skipping GIF frame for {r['path']}: Less than 2 bands available.")
                    continue
                vv_db, vh_db = arr[0], arr[1]
                frames.append(make_rvi_frame(vv_db, vh_db, ds=GIF_DOWNSAMPLE))
                labels.append(f"{r['year']}_{r['quart']}")
            except Exception as e:
                 print(f"Unexpected error creating GIF frame for {r['path']}: {e}") # More specific error message
                 continue # Skip this frame but continue with others

        if frames:
            gif_local = f"/content/out/gifs/ao{a}_rvi.gif"
            imageio.mimsave(gif_local, frames, duration=0.8)
            # Replace gcs_upload with save_to_drive
            save_to_drive(gif_local)
            print(f"🎞️ GIF saved for AOI {a}")

# ============================================================
# 8) Autoencoder Model
# ============================================================

# Define IN_CHANNELS based on SELECT_BANDS
IN_CHANNELS = len(SELECT_BANDS)

# Simple Conv-ReLU-BatchNorm block
def conv_block(in_c, out_c):
    return nn.Sequential(
        nn.Conv2d(in_c, out_c, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.BatchNorm2d(out_c)
    )

# Contracting Path (Encoder)
def encoder_block(in_c, out_c):
    return nn.Sequential(
        conv_block(in_c, out_c),
        conv_block(out_c, out_c),
        nn.MaxPool2d(2)
    )

# Expanding Path (Decoder)
def decoder_block(in_c, out_c):
    return nn.Sequential(
        nn.ConvTranspose2d(in_c, out_c, kernel_size=2, stride=2),
        conv_block(out_c, out_c),
        conv_block(out_c, out_c)
    )

class UNetAutoencoder(nn.Module):
    def __init__(self, in_channels=IN_CHANNELS, base_channels=BASE_CHANNELS):
        super().__init__()
        self.enc1 = encoder_block(in_channels, base_channels)
        self.enc2 = encoder_block(base_channels, base_channels*2)
        self.enc3 = encoder_block(base_channels*2, base_channels*4)
        self.enc4 = encoder_block(base_channels*4, base_channels*8)

        self.bottleneck = conv_block(base_channels*8, base_channels*16)

        self.dec4 = decoder_block(base_channels*16, base_channels*8)
        self.dec3 = decoder_block(base_channels*8, base_channels*4)
        self.dec2 = decoder_block(base_channels*4, base_channels*2)
        self.dec1 = decoder_block(base_channels*2, base_channels)

        self.out_conv = nn.Conv2d(base_channels, in_channels, kernel_size=1)

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(enc1)
        enc3 = self.enc3(enc2)
        enc4 = self.enc4(enc3)

        bottleneck = self.bottleneck(enc4)

        dec4 = self.dec4(bottleneck)
        dec3 = self.dec3(dec4 + enc3) # Skip connection
        dec2 = self.dec2(dec3 + enc2) # Skip connection
        dec1 = self.dec1(dec2 + enc1) # Skip connection

        return self.out_conv(dec1)

# Instantiate model
model = UNetAutoencoder(in_channels=IN_CHANNELS, base_channels=BASE_CHANNELS)

Found 93 rasters across baseline + hotspots (first 5):
[{'aoi': 1, 'year': 2020, 'quart': 'Q1', 'path': 'CLEAN/Interest_clean/AOI_1_2020_Q1.tif', 'source': 'hotspot'}, {'aoi': 1, 'year': 2020, 'quart': 'Q2', 'path': 'CLEAN/Interest_clean/AOI_1_2020_Q2.tif', 'source': 'hotspot'}, {'aoi': 1, 'year': 2020, 'quart': 'Q3', 'path': 'CLEAN/Interest_clean/AOI_1_2020_Q3.tif', 'source': 'hotspot'}, {'aoi': 1, 'year': 2020, 'quart': 'Q4', 'path': 'CLEAN/Interest_clean/AOI_1_2020_Q4.tif', 'source': 'hotspot'}, {'aoi': 1, 'year': 2021, 'quart': 'Q1', 'path': 'CLEAN/Interest_clean/AOI_1_2021_Q1.tif', 'source': 'hotspot'}]
Computing metrics per raster...
ℹ️ Using default band mapping (B1→VV, B2→VH, B3→angle) for AOI_1_2020_Q1.tif
ℹ️ Using default band mapping (B1→VV, B2→VH, B3→angle) for AOI_1_2020_Q2.tif
ℹ️ Using default band mapping (B1→VV, B2→VH, B3→angle) for AOI_1_2020_Q3.tif
ℹ️ Using default band mapping (B1→VV, B2→VH, B3→angle) for AOI_1_2020_Q4.tif
ℹ️ Using default band mapping (B1→VV, B2→VH,

In [ ]:
# Unmount if drive got stuck before
from google.colab import drive
!fusermount -u /content/drive || true

# Mount again cleanly
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import rasterio
import numpy as np
import csv, os, requests
from io import BytesIO
from google.colab import drive


BUCKET = "nasa_sar_spacetron"
OUT_DIR = "/content/drive/MyDrive/SAR_Results"
os.makedirs(OUT_DIR, exist_ok=True)

# === your raster paths from GCS ===
raster_paths = [
    "CLEAN/Interest_clean/AOI_1_2024_Q1.tif",
    "CLEAN/Interest_clean/AOI_2_2023_Q2.tif",
    "CLEAN/Interest_clean/AOI_3_2022_Q4.tif",
    "CLEAN/Interest_clean/AOI_4_2025_Q1.tif",
]

records = []
for path in raster_paths:
    url = f"https://storage.googleapis.com/{BUCKET}/{path}"
    print(f"\n📂 Processing {url}")
    try:
        resp = requests.get(url, stream=True)
        resp.raise_for_status()
        with rasterio.open(BytesIO(resp.content)) as src:
            arr = src.read()[:3]     # first 3 bands → VV, VH, angle
            if np.all(np.isnan(arr)) or np.nanmean(arr) == 0:
                print("⚠️ Skipping (empty or no signal)")
                continue

            vv, vh, ang = arr[0], arr[1], arr[2]
            rec = {
                "file": os.path.basename(path),
                "aoi": path.split("_")[1],
                "year": path.split("_")[2],
                "quarter": path.split("_")[3].split(".")[0],
                "vv_mean": np.nanmean(vv),
                "vv_std": np.nanstd(vv),
                "vh_mean": np.nanmean(vh),
                "vh_std": np.nanstd(vh),
                "angle_mean": np.nanmean(ang),
                "angle_std": np.nanstd(ang),
                "valid_ratio": np.sum(~np.isnan(arr)) / arr.size
            }
            records.append(rec)
            print(f"✅ Added metrics for {rec['file']}")

    except Exception as e:
        print(f"❌ Error reading {path}: {e}")

# === Save metrics to Drive ===
if records:
    out_csv = os.path.join(OUT_DIR, "sar_metrics_valid.csv")
    with open(out_csv, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=records[0].keys())
        writer.writeheader(); writer.writerows(records)
    print(f"\n✅ CSV saved to Drive: {out_csv}")
else:
    print("❌ No valid rasters processed.")



📂 Processing https://storage.googleapis.com/nasa_sar_spacetron/CLEAN/Interest_clean/AOI_1_2024_Q1.tif
✅ Added metrics for AOI_1_2024_Q1.tif

📂 Processing https://storage.googleapis.com/nasa_sar_spacetron/CLEAN/Interest_clean/AOI_2_2023_Q2.tif
✅ Added metrics for AOI_2_2023_Q2.tif

📂 Processing https://storage.googleapis.com/nasa_sar_spacetron/CLEAN/Interest_clean/AOI_3_2022_Q4.tif
✅ Added metrics for AOI_3_2022_Q4.tif

📂 Processing https://storage.googleapis.com/nasa_sar_spacetron/CLEAN/Interest_clean/AOI_4_2025_Q1.tif
✅ Added metrics for AOI_4_2025_Q1.tif

✅ CSV saved to Drive: /content/drive/MyDrive/SAR_Results/sar_metrics_valid.csv


In [ ]:
!find /content -type f -name "*.csv"


/content/drive/MyDrive/SAR_Exports/asf-opera-displacement-2025-10-04_10-26-41.csv
/content/drive/MyDrive/SAR_Exports/sar_exports_manifest.csv
/content/drive/MyDrive/SAR_Results/sar_metrics_valid.csv
/content/out/sar_metrics.csv
/content/out/recovery_summaries.csv
/content/sample_data/california_housing_train.csv
/content/sample_data/mnist_test.csv
/content/sample_data/mnist_train_small.csv
/content/sample_data/california_housing_test.csv


In [ ]:
import shutil, os

src = "/content/out/sar_metrics.csv"
dst = "/content/drive/MyDrive/SAR_Results/sar_metrics_valid.csv"

os.makedirs("/content/drive/MyDrive/SAR_Results", exist_ok=True)
if os.path.exists(src):
    shutil.copy(src, dst)
    print(f"✅ Copied to Drive: {dst}")
else:
    print(f"⚠️ Source not found: {src}")


✅ Copied to Drive: /content/drive/MyDrive/SAR_Results/sar_metrics_valid.csv


In [ ]:
!rm -rf /content/drive  # 🧹 Delete the old mount folder


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import shutil, os

src = "/content/sar_metrics_valid.csv"
dst = "/content/drive/MyDrive/SAR_Results/sar_metrics_valid.csv"

os.makedirs("/content/drive/MyDrive/SAR_Results", exist_ok=True)
if os.path.exists(src):
    shutil.copy(src, dst)
    print(f"✅ CSV copied to Drive: {dst}")
else:
    print("⚠️ No CSV found at", src)


⚠️ No CSV found at /content/sar_metrics_valid.csv
